In [1]:
# Notebook 출력설정
# 주요 라이브러리 임포트

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = True
import seaborn as sns

In [2]:
DIR_DATASET = "C:/Users/0stix/Datasets/"
NAME_PROJECT = '0000-kaggle-titanic'

In [21]:
df_train = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/train.csv', index_col='PassengerId')
df_test = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/test.csv', index_col='PassengerId')

len_train = len(df_train)
df_all = pd.concat([df_train, df_test], axis=0)
target_ = 'Survived'

In [4]:
df_v0 = pd.DataFrame()
df_v0['Sex'] = df_all['Sex'].astype('category').cat.codes
df_v0['Embarked'] = df_all['Embarked'].fillna('S').astype('category').cat.codes
df_v0['SibSp'] = df_all['SibSp'].copy()
df_v0['Parch'] = df_all['Parch'].copy()
df_v0['Family'] = 1 + df_v0['SibSp'] + df_v0['Parch']
df_v0['Solo'] = df_v0['Family'].apply(lambda x:1 if x == 1 else 0)
df_v0['Fare'] = pd.qcut(df_all['Fare'], 5).astype('category').cat.codes

df_v0['Title'] = df_all['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_v0['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other',inplace=True)
df_v0['Title'].replace('Mlle', 'Miss')
df_v0['Title'].replace('Ms', 'Miss')
df_v0['Title'].replace('Mme', 'Mrs')
df_v0['Title'] = df_v0['Title'].astype('category').cat.codes

df_v0['Age'] = df_all['Age'].copy()
df_v0['Age'].fillna(df_v0.groupby("Title")["Age"].transform("median"), inplace=True)

cnt_ = 1
lst_age = [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 120]
for age_a, age_b in zip(lst_age[:-1],lst_age[1:]):
    df_v0.loc[(df_v0['Age'] > age_a) & (df_v0['Age'] <= age_b), 'Age_c'] = cnt_
    cnt_ += 1

df_v0['Pclass'] = df_all['Pclass'].copy()
dct_ = {chr_:idx_ for idx_, chr_ in enumerate('ABCDEFGT')}
df_v0['Cabin'] = df_all['Cabin'].str[:1].map(dct_)
df_v0['Cabin'] = df_v0.groupby('Pclass')['Cabin'].transform('median')

df_v0['Survived'] = df_all['Survived'].copy()
# df_train_v0 = df_v0[:len_train]
# df_test_v0 = df_v0[len_train:]

In [5]:
X_, y_ = df_v0.drop(['Survived'], axis=1).values, df_v0[['Survived']].values
# X_, y_ = df_v0.drop(['Survived'], axis=1), df_v0[['Survived']]

X_train, X_test, y_train = X_[:len_train], X_[len_train:], y_[:len_train]

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import time


def objective_(trial_):
    lst_param = {
        "random_state":trial_.suggest_categorical("random_state", [1]),
        'learning_rate' : trial_.suggest_loguniform('learning_rate', 0.0001, 0.3),
        'bagging_temperature' :trial_.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators": 1000,
        "max_depth":trial_.suggest_int("max_depth", 2, 9),
        'random_strength' :trial_.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg":trial_.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial_.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial_.suggest_int("max_bin", 50, 200),
        'od_type': trial_.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        # 'task_type': trial_.suggest_categorical('task_type', ['GPU']),
        # 'loss_function': trial_.suggest_categorical('loss_function', ['Accuracy']),
        # 'eval_metric': trial_.suggest_categorical('eval_metric', ['Accuracy'])
    }
    
    X_tr_t, X_va_t, y_tr_t, y_va_t = train_test_split(X_train, y_train, test_size=.3, random_state=1)
    
    model_ = CatBoostClassifier(**lst_param)
    model_.fit(
        X_tr_t,
        y_tr_t,
        eval_set=[(X_va_t, y_va_t)],
        early_stopping_rounds=15,
        verbose=0
    )
    
    y_tr_p = model_.predict(X_tr_t)
    y_va_p = model_.predict(X_va_t)
    score_tr = accuracy_score(y_tr_t, y_tr_p)
    score_va = accuracy_score(y_va_t, y_va_p)
    print(score_tr, score_va)
    
    return score_va

In [7]:
import optuna
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

TRIALS = 500
TIMEOUT = 1800

sampler_ = TPESampler(seed=1)

study_ = optuna.create_study(
    direction='maximize',
    sampler=sampler_,
)

study_.optimize(
    objective_, 
    n_trials=TRIALS, 
    timeout=TIMEOUT, 
    n_jobs=1, 
    callbacks=None,
    # show_progress_bar=True,
    )

print(study_.best_value)
print(study_.best_trial.params)

[I 2022-03-27 18:54:33,679] A new study created in memory with name: no-name-928d013e-8255-4b4a-a89e-bfcad490a823
[I 2022-03-27 18:54:34,219] Trial 0 finished with value: 0.7611940298507462 and parameters: {'random_state': 1, 'learning_rate': 0.0028186170601294433, 'bagging_temperature': 7.608481233714797, 'max_depth': 2, 'random_strength': 30, 'l2_leaf_reg': 4.41120916560522e-06, 'min_child_samples': 13, 'max_bin': 78, 'od_type': 'Iter'}. Best is trial 0 with value: 0.7611940298507462.


0.8346709470304976 0.7611940298507462


[I 2022-03-27 18:54:34,443] Trial 1 finished with value: 0.7835820895522388 and parameters: {'random_state': 1, 'learning_rate': 0.007473621824976351, 'bagging_temperature': 0.47509237210306143, 'max_depth': 7, 'random_strength': 20, 'l2_leaf_reg': 2.6344741917364455e-05, 'min_child_samples': 7, 'max_bin': 151, 'od_type': 'Iter'}. Best is trial 1 with value: 0.7835820895522388.


0.8860353130016051 0.7835820895522388


[I 2022-03-27 18:54:35,339] Trial 2 finished with value: 0.7611940298507462 and parameters: {'random_state': 1, 'learning_rate': 0.0003077105652918673, 'bagging_temperature': 0.0620020367716472, 'max_depth': 8, 'random_strength': 97, 'l2_leaf_reg': 9.409591102995692e-06, 'min_child_samples': 71, 'max_bin': 182, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7835820895522388.


...

0.8571428571428571 0.7947761194029851


[I 2022-03-27 18:59:19,473] Trial 498 finished with value: 0.7873134328358209 and parameters: {'random_state': 1, 'learning_rate': 0.001228441960638885, 'bagging_temperature': 0.11903742719079842, 'max_depth': 7, 'random_strength': 2, 'l2_leaf_reg': 9.79658161649502e-06, 'min_child_samples': 78, 'max_bin': 147, 'od_type': 'Iter'}. Best is trial 194 with value: 0.8022388059701493.


0.8908507223113965 0.7873134328358209


[I 2022-03-27 18:59:20,404] Trial 499 finished with value: 0.7873134328358209 and parameters: {'random_state': 1, 'learning_rate': 0.000646754207269184, 'bagging_temperature': 0.13790634878881858, 'max_depth': 8, 'random_strength': 4, 'l2_leaf_reg': 1.1845811254610167e-05, 'min_child_samples': 64, 'max_bin': 139, 'od_type': 'Iter'}. Best is trial 194 with value: 0.8022388059701493.


0.8828250401284109 0.7873134328358209
0.8022388059701493
{'random_state': 1, 'learning_rate': 0.0007046467385886047, 'bagging_temperature': 0.12200640988518986, 'max_depth': 7, 'random_strength': 1, 'l2_leaf_reg': 1.2861408231357856e-05, 'min_child_samples': 58, 'max_bin': 143, 'od_type': 'Iter'}


In [11]:
from catboost import CatBoostClassifier

best_params = study_.best_params

X_tr_t, X_va_t, y_tr_t, y_va_t = train_test_split(X_train, y_train, test_size=0.3, random_state=1)
model_tmp = CatBoostClassifier(**best_params, n_estimators=30000, verbose=1000)
model_tmp.fit(X_tr_t, y_tr_t, eval_set=[(X_va_t, y_va_t)], early_stopping_rounds=15)

0:	learn: 0.6924198	test: 0.6925170	best: 0.6925170 (0)	total: 872us	remaining: 26.2s
1000:	learn: 0.3570328	test: 0.4719808	best: 0.4719808 (1000)	total: 639ms	remaining: 18.5s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 0.4701055037
bestIteration = 1116

Shrink model to first 1117 iterations.


In [12]:
model_final = CatBoostClassifier(**best_params, 
                                 n_estimators=model_tmp.get_best_iteration(), 
                                 verbose=1000)
model_final.fit(X_train, y_train)
y_pred = model_final.predict(X_test)

0:	learn: 0.6924927	total: 17.3ms	remaining: 19.3s
1000:	learn: 0.3966805	total: 717ms	remaining: 82.3ms
1115:	learn: 0.3843078	total: 796ms	remaining: 0us


In [22]:
df_sub = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/gender_submission.csv', index_col='PassengerId')
df_sub[target_] = y_pred.astype(int)

In [26]:
import datetime
now = datetime.datetime.now()
str_datetime = now.strftime("%y%m%d_%H%M%S")
df_sub.to_csv(DIR_DATASET + 'submission-' + NAME_PROJECT + '-'+str_datetime+'.csv', index = 1)